In [1]:
import os
from typing import Iterable, Dict

import pandas as pd
import numpy as np

sample = 'tz_opendata_z01012021_po01072021.csv'

## What need to be defined before start processing data

### Tables

- simple table: has no dependency on other tables. have only primary key, which can be a foreign key for some other tables.

- complex table: has dependency on other tables. some fields can be represented just by primary key of other table (not only simple), but also can contain some data, which is not represented in other tables.

In [2]:
#  Tables configuration
# simle tables
brand = ['brand']
model = ['model']
fuel = ['fuel']
kind = ['kind']
bodytype = ['body']
purpose = ['purpose']
license_plates = ['license_plates']
owners = ['person']
employee = ['employee', 'dep']
dep = ['dep', 'reg_addr_koatuu']

# complex tables
engine = ['capacity', *fuel]
operations = ['oper_code', 'oper_name', *dep]
vehicle = [*brand, *model, *engine, *kind, *
           bodytype, *purpose, 'own_weight', 'total_weight']
cars_for_registration = [*vehicle, 'vin', 'make_year', 'color']
registration_activities = [*operations, *
                           cars_for_registration, *owners, *license_plates]

In [3]:
# put all tables into dict to easily
tables_config = {
    "brand": ['brand'],
    "model": ['model'],
    "fuel": ['fuel'],
    "kind": ['kind'],
    "bodytype": ['body'],
    "purpose": ['purpose'],
    "license_plates": ['license_plates'],
    "owners": ['person'],
    "employee": ['employee', 'dep'],
    "dep": ['dep', 'reg_addr_koatuu'],
    "engine": ['capacity', 'fuel'],
    "operations": ['oper_code', 'oper_name', 'dep', 'reg_addr_koatuu'],
    "cars_for_registration": ['brand', 'model', 'capacity', 'fuel', 'kind', 'body', 'purpose', 'own_weight', 'total_weight' 'vin', 'make_year', 'color'],
    "vehicle": ['brand', 'model', 'capacity', 'fuel', 'kind', 'body', 'purpose', 'own_weight', 'total_weight'],
    "registration_activities": [
        'oper_code', 'oper_name', 'dep', 'reg_addr_koatuu', 'brand', 'model', 'capacity', 'fuel', 'kind', 'body', 'purpose', 'own_weight', 'total_weight' 'vin', 'make_year', 'color''person', 'license_plates'
    ]
}

### Mappers

- dtypes: dictionary with column name and corresponding dtype to parse dtypes while reading data

In [4]:
dtypes_mapper = {
    'person': 'category',
    'fuel': 'category',
    'kind': 'category',
    'purpose': 'category',
    'reg_addr_koatuu': 'Int64',
    'oper_code': 'Int64',
    'employee': 'Int64',
    'make_year': 'Int64',
    'capacity': 'Int64',
    'own_weight': 'Int64',
    'total_weight': 'Int64'
}
columns_mapper = {
    'dep_code': 'employee',
    'n_reg_new': 'license_plates'
}
# na_values_mapper = {
#     'reg_addr_koatuu': 0
# }

### Helpers

- func to split input data to tables
- func to add table-based id and additional DataFrame attr **_name_**
- any other logic, which need to be implemented but not covered with __pandas__ functionality

In [17]:
def split_to_tables(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = (
        df[cols]
        .drop_duplicates()
        .reset_index()
        .drop('index', axis=1)
        .rename(mapper={x: x.lower() for x in cols}, axis=1)
    )
    return df


def add_attrs_to_table(df: pd.DataFrame, name: str) -> pd.DataFrame:
    # df[f'{name}_id'] = ((name[:3] if len(name) >= 3 else name) + 
    #                     pd.Series(df.index).astype("str"))
    df[f'{name}_id'] = df.index
    df.__setattr__('name', name)
    return df


def map_and_drop_cols(mapped: pd.DataFrame, mapper: pd.DataFrame, tables_config: Dict) -> pd.DataFrame:
    on = tables_config.get(mapper.name)
    try:
        return mapped.merge(mapper, how='left', on=on).drop(on, axis=1)
    except KeyError:
        return mapped.merge(mapper, how='left').drop(on, axis=1)

def map_tables(table_to_map: str, map_with: Iterable, tables: Dict, tables_config: Dict) -> pd.DataFrame:
    df = tables.get(table_to_map)
    mappers = [tables.get(x) for x in map_with]
    orig_name = df.name
    for mapper in mappers:
        df = map_and_drop_cols(df, mapper, tables_config)
    df.__setattr__('name', orig_name)
    return df

##  Data processing

### Quick check input data

read a few rows to understand how data looks like and what transformations or mapping can be done while reading

In [6]:
# columns
df_exmpl = pd.read_csv(
    sample,
    sep=';',
    nrows=5
)
df_exmpl

,PERSON,REG_ADDR_KOATUU,OPER_CODE,OPER_NAME,D_REG,DEP_CODE,DEP,BRAND,MODEL,VIN,MAKE_YEAR,COLOR,KIND,BODY,PURPOSE,FUEL,CAPACITY,OWN_WEIGHT,TOTAL_WEIGHT,N_REG_NEW
0,P,6.310137e+09,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,02.04.2021,12355,ТСЦ 6341,PEUGEOT,3008,VF30U9HR8BS338559,2011,ЧОРНИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,1560,1423,2030,АХ5957КЕ
1,P,8.036100e+09,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,06.04.2021,13960,ТСЦ 8048,TOYOTA,COROLLA,JTNBV56E70J182361,2012,СІРИЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН,1598,1250,1760,КА5612СА
2,P,3.510100e+09,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,19.02.2021,12290,ТСЦ 8041,LAND ROVER,DISCOVERY,SALLAAAF4CA632784,2012,СИНІЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,2993,2643,3240,ВА2719СО
3,P,NaN,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,10.04.2021,13960,ТСЦ 8048,RENAULT,KOLEOS,VF1VY0C0VUC394069,2011,СІРИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,БЕНЗИН,2488,1750,2250,КА4200СІ
4,P,NaN,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,18.05.2021,12381,ТСЦ 7145,NISSAN,NOTE,SJNFAAE11U2161945,2012,СИНІЙ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,БЕНЗИН,1386,1164,1546,СА4275ІС


In [18]:
df_header = list(
    pd.read_csv(
        sample,
        sep=';',
        nrows=0
    ).rename(columns=str.lower).rename(mapper=columns_mapper, axis=1)
)

df_exmpl = pd.read_csv(
    sample,
    sep=';',
    nrows=50000,
    on_bad_lines='skip',  # to prevent errors while reading
    dtype=dtypes_mapper,  # map dtypes
    # na_values=na_values_mapper,  # didnt work properly
    header=0,
    names=df_header,  # lets use already formated columns instead of renaming after
    parse_dates=['d_reg']
)
df_exmpl.head()

,person,reg_addr_koatuu,oper_code,oper_name,d_reg,employee,dep,brand,model,vin,make_year,color,kind,body,purpose,fuel,capacity,own_weight,total_weight,license_plates
0,P,6310136600,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-02-04,12355,ТСЦ 6341,PEUGEOT,3008,VF30U9HR8BS338559,2011,ЧОРНИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,1560,1423,2030,АХ5957КЕ
1,P,8036100000,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-06-04,13960,ТСЦ 8048,TOYOTA,COROLLA,JTNBV56E70J182361,2012,СІРИЙ,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,БЕНЗИН,1598,1250,1760,КА5612СА
2,P,3510100000,310,ПЕРЕРЕЄСТРАЦІЯ НА НОВОГО ВЛАСНИКА ЗА ДОГ. КУП....,2021-02-19,12290,ТСЦ 8041,LAND ROVER,DISCOVERY,SALLAAAF4CA632784,2012,СИНІЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,ДИЗЕЛЬНЕ ПАЛИВО,2993,2643,3240,ВА2719СО
3,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-10-04,13960,ТСЦ 8048,RENAULT,KOLEOS,VF1VY0C0VUC394069,2011,СІРИЙ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,БЕНЗИН,2488,1750,2250,КА4200СІ
4,P,<NA>,315,ПЕРЕРЕЄСТРАЦІЯ ТЗ НА НОВ. ВЛАСН. ПО ДОГОВОРУ У...,2021-05-18,12381,ТСЦ 7145,NISSAN,NOTE,SJNFAAE11U2161945,2012,СИНІЙ,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,БЕНЗИН,1386,1164,1546,СА4275ІС


### Split dataframe to our tables and add enrich them with some data

In [19]:
df = df_exmpl.copy()  # lets make a copy

reg_activities_mapping = ('registration_activities', ['cars_for_registration', 'operations', 'owners', 'license_plates'])
vehicle_mapping = ('vehicle', ['brand', 'model', 'engine', 'kind', 'bodytype', 'purpose'])
operations_mapping = ('operations', ['dep'])
emp_mapping = ('employee', ['dep'])
dep_mapping = ('dep', ['employee'])
engine_mapping = ('engine', ['fuel'])
car_for_reg_mapping = ('cars_for_registration', ['vehicle'])

tables = {}
mapped_tables = {}

for k, v in tables_config.items():
    tables[k] = add_attrs_to_table(split_to_tables(df, v), k)
    
for mapping in [engine_mapping, dep_mapping, emp_mapping, car_for_reg_mapping,
                operations_mapping, vehicle_mapping, reg_activities_mapping]:
    mapped_tables[mapping[0]] = map_tables(*mapping, tables)

In [76]:
mapped_tables.get('vehicle')

,own_weight,total_weight,vehicle_id,brand_id,model_id,engine_id,kind_id,bodytype_id,purpose_id
0,1423,2030,0,0,0,0,0,0,0
1,1250,1760,1,1,1,1,0,1,0
2,2643,3240,2,2,2,2,0,0,0
3,1750,2250,3,3,3,3,0,0,0
4,1164,1546,4,4,4,4,0,2,0
...,...,...,...,...,...,...,...,...,...
38150,1250,1670,38150,13,209,88,0,1,0
38151,1690,2170,38151,13,56,24,0,0,0
38152,8150,18000,38152,21,2895,219,1,9,1
38153,1910,2720,38153,42,306,366,0,0,0


In [97]:
from datetime import datetime

In [135]:
datetime.now().strftime('%m%d_%S_%f')

'0717_280930'

In [79]:
tables.update(mapped_tables)

In [82]:
tables.get('vehicle') == mapped_tables.get('vehicle')

,own_weight,total_weight,vehicle_id,brand_id,model_id,engine_id,kind_id,bodytype_id,purpose_id
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
38150,True,True,True,True,True,True,True,True,True
38151,True,True,True,True,True,True,True,True,True
38152,True,True,True,True,True,True,True,True,True
38153,True,True,True,True,True,True,True,True,True


,brand,model,capacity,fuel,kind,body,purpose,own_weight,total_weight,vehicle_id
0,PEUGEOT,3008,1560,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1423,2030,0
1,TOYOTA,COROLLA,1598,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1760,1
2,LAND ROVER,DISCOVERY,2993,ДИЗЕЛЬНЕ ПАЛИВО,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,2643,3240,2
3,RENAULT,KOLEOS,2488,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1750,2250,3
4,NISSAN,NOTE,1386,БЕНЗИН,ЛЕГКОВИЙ,ХЕТЧБЕК,ЗАГАЛЬНИЙ,1164,1546,4
...,...,...,...,...,...,...,...,...,...,...
38150,VOLKSWAGEN,BORA,1600,БЕНЗИН,ЛЕГКОВИЙ,СЕДАН,ЗАГАЛЬНИЙ,1250,1670,38150
38151,VOLKSWAGEN,PASSAT,1390,БЕНЗИН АБО ГАЗ,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1690,2170,38151
38152,MAN,TGA 18.463,12816,ДИЗЕЛЬНЕ ПАЛИВО,ВАНТАЖНИЙ,СІДЛОВИЙ ТЯГАЧ,СПЕЦІАЛІЗОВАНИЙ,8150,18000,38152
38153,LEXUS,LX 570,5663,БЕНЗИН,ЛЕГКОВИЙ,УНІВЕРСАЛ,ЗАГАЛЬНИЙ,1910,2720,38153


In [ ]:
os.makedirs()

In [73]:
mapped_tables.get('registration_activities').to_csv('reg.csv', index=False)
tables.get('registration_activities').to_csv('reg2.csv', index=False)

In [69]:
df1[['own_weight']][(df1['own_weight'].isna()) == True]

,own_weight
3004,<NA>
4252,<NA>
4887,<NA>
9565,<NA>
12766,<NA>
13028,<NA>
13614,<NA>
15623,<NA>
16957,<NA>
21255,<NA>


In [96]:
df_exmpl = pd.read_csv(
    sample,
    sep=';',
#     chunksize=50000,
    nrows=250000,
    on_bad_lines='skip',  # to prevent errors while reading
    error_bad_lines=False,
    dtype=dtypes_mapper,  # map dtypes
    # na_values=na_values_mapper,  # didnt work properly
    header=0,
    names=df_header,  # lets use already formated columns instead of renaming after
#     parse_dates=['d_reg']
)

ValueError: Both on_bad_lines and error_bad_lines/warn_bad_lines are set. Please only set on_bad_lines.

In [93]:
next(df_exmpl)

ValueError: Unable to parse string "80,5" at position 25043

In [ ]:
from typing import Iterable, Dict

def map_tables2(table_to_map: str, map_with: Iterable, tables: Dict) -> pd.DataFrame:
    def map_and_drop_cols(mapped: pd.DataFrame, mapper: pd.DataFrame) -> pd.DataFrame:
        on = tables_config.get(mapper.name)
        return mapped.merge(mapper, how='left').drop(on, axis=1)
    
    df = tables.get(table_to_map)
    df_name = df.name
    mappers = [tables.get(x) for x in map_with]
    for mapper in mappers:
        df = map_and_drop_cols(df, mapper)
    df.__setattr__('name', df_name)
    return df

In [ ]:
def map_and_drop_cols(mapped: pd.DataFrame, mapper: pd.DataFrame) -> pd.DataFrame:
        on = tables_config.get(mapper.name)
        return mapped.merge(mapper, how='left', on=on).drop(on, axis=1)